In [62]:
# Cell 1 — Imports & configuration
import os
import shutil
import subprocess
import time
import pandas as pd
from yt_dlp import YoutubeDL

OUTPUT_DIR   = "data_set_try1/videos"                             # dossier de sortie
CSV_LABELS   = "data_set_try1/dataset/labels_semantic_clusters.csv"
CSV_DATA     = "data_set_try1/dataset/vggsound.csv"
CLUSTER      = '3'                                  # numéro de cluster
DURATION_SEC = 10                                   # durée du clip en secondes

# Vérifions qu’on a bien ffmpeg dispo
if shutil.which("ffmpeg") is None:
    raise RuntimeError("⚠️ ffmpeg n'est pas installé — indispensable pour ce script.")

In [63]:
# Cell 2 — Fonctions utilitaires
def sec_to_timestamp(s: int) -> str:
    """Convertit un entier de secondes en 'HH:MM:SS'."""
    return time.strftime("%H:%M:%S", time.gmtime(s))

def download_segment_fast(video_id: str, start_sec: int, duration: int, dest_folder: str):
    """
    1) Récupère l’URL du flux MP4 (360p, conteneur mp4).
    2) Lance ffmpeg avec -ss et -t pour ne tirer que la portion souhaitée.
    """
    os.makedirs(dest_folder, exist_ok=True)
    url = f"https://www.youtube.com/watch?v={video_id}"

    # 1) Extraction de l’URL directe du flux MP4 (audio+vidéo) H264≤360p
    ydl_opts = {
        'format': 'best[height<=360][ext=mp4]',
        'quiet': True,
        'skip_download': True,
    }
    with YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=False)
        stream_url = info['url']

    # 2) Construction du nom et des timestamps
    t0 = sec_to_timestamp(start_sec)
    output_name = f"{video_id}_{start_sec}s.mp4"
    output_path = os.path.join(dest_folder, output_name)

    # 3) Appel ffmpeg : -ss avant -i pour seek rapide, -t pour durée, -c copy pour pas ré-encoder
    cmd = [
        'ffmpeg',
        '-ss', t0,
        '-i', stream_url,
        '-t', str(duration),
        '-c', 'copy',
        output_path
    ]
    subprocess.run(cmd, check=True)

def download_segment_crop256(video_id: str, start_sec: int, duration: int, dest_folder: str):
    """
    1) Récupère l’URL du flux MP4 (360p).
    2) Coupe le segment [start_sec, start_sec+duration].
    3) Croppe en 256×256 centré.
    4) Sort en MP4 (H.264+AAC).
    """
    os.makedirs(dest_folder, exist_ok=True)
    url = f"https://www.youtube.com/watch?v={video_id}"

    # Extraction de l’URL directe du flux MP4
    ydl_opts = {'format': 'best[height<=360][ext=mp4]', 'quiet': True, 'skip_download': True}
    with YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=False)
        stream_url = info['url']

    # Timestamp de début
    t0 = sec_to_timestamp(start_sec)
    output_name = f"{video_id}_{start_sec}s_256crop.mp4"
    output_path = os.path.join(dest_folder, output_name)

    # ffmpeg : -ss avant -i pour seek rapide, -t pour durée, -vf pour crop, 
    # -c:v libx264 pour encoder, -preset veryfast pour vitesse, -crf 23 qualité raisonnable, -c:a copy
    cmd = [
        'ffmpeg',
        '-ss', t0,
        '-i', stream_url,
        '-t', str(duration),
        '-vf', 'crop=256:256:(in_w-256)/2:(in_h-256)/2',
        '-c:v', 'libx264',
        '-preset', 'veryfast',
        '-crf', '23',
        '-c:a', 'copy',
        output_path
    ]
    subprocess.run(cmd, check=True)



In [67]:
# Cell 3 — Lecture & filtrage du dataset
labels_df = pd.read_csv(CSV_LABELS, header=None,
                        names=['label','count','cluster'])
used_labels = labels_df[labels_df['cluster']==CLUSTER]['label'].values

data_df = pd.read_csv(CSV_DATA, header=None,
                      names=['video_id','start_sec','label','split'])
sub_df = data_df[data_df['label'].isin(used_labels)].sample(10, random_state=42)

In [68]:
labels_df

,label,count,cluster
0,label,count,cluster
1,fireworks banging,1050,1
2,tap dancing,1050,2
3,"motorboat, speedboat acceleration",1050,0
4,playing trombone,1050,2
...,...,...,...
306,snake rattling,199,1
307,zebra braying,199,1
308,playing castanets,199,3
309,cow lowing,199,1


In [47]:
# Cell 4 — Boucle de téléchargement
for vid, start, label, split in sub_df.itertuples(index=False):
    real_end = min(start + DURATION_SEC, start + DURATION_SEC)  # juste pour l'affichage
    print(f"▶️ {vid} @ {start}s → +{DURATION_SEC}s …")
    try:
        download_segment_crop256(vid, int(start), DURATION_SEC, OUTPUT_DIR)
    except Exception as e:
        print(f"⚠️ Erreur pour {vid}@{start}s : {e}")

print("✅ Tous les segments (10 s) sont prêts en MP4 dans", OUTPUT_DIR)

▶️ 5PVKgUGg0LQ @ 450s → +10s …


ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with clang version 17.0.6
  configuration: --prefix=/opt/anaconda3/envs/dataloader --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --enable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1728332276747/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --disable-gnutls --enable-libmp3lame --enable-libvpx --enable-libass --enable-pthreads --enable-libopenvino --enable-gpl --enable-libx264 --enable-libx265 --enable-libaom --enable-libsvtav1 --enable-libxml2 --enable-pic --enable-shared --disable-static --enable-version3 --enable-zlib --enable-lib

▶️ wd2RVda164o @ 30s → +10s …


ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with clang version 17.0.6
  configuration: --prefix=/opt/anaconda3/envs/dataloader --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --enable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1728332276747/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --disable-gnutls --enable-libmp3lame --enable-libvpx --enable-libass --enable-pthreads --enable-libopenvino --enable-gpl --enable-libx264 --enable-libx265 --enable-libaom --enable-libsvtav1 --enable-libxml2 --enable-pic --enable-shared --disable-static --enable-version3 --enable-zlib --enable-lib

▶️ IpHCwR7ACjI @ 189s → +10s …


...

ERROR: [youtube] OVBQuUJhtfg: Video unavailable


⚠️ Erreur pour OVBQuUJhtfg@30s : ERROR: [youtube] OVBQuUJhtfg: Video unavailable
▶️ CeTbzVsGfRw @ 30s → +10s …


ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with clang version 17.0.6
  configuration: --prefix=/opt/anaconda3/envs/dataloader --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --enable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1728332276747/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --disable-gnutls --enable-libmp3lame --enable-libvpx --enable-libass --enable-pthreads --enable-libopenvino --enable-gpl --enable-libx264 --enable-libx265 --enable-libaom --enable-libsvtav1 --enable-libxml2 --enable-pic --enable-shared --disable-static --enable-version3 --enable-zlib --enable-lib

✅ Tous les segments (10 s) sont prêts en MP4 dans data_set_try1/videos


Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'https://rr1---sn-nfpuji-1gil.googlevideo.com/videoplayback?expire=1745868036&ei=pIAPaOKKNtPO6dsPo9K1-Ao&ip=2001%3A620%3A618%3A5c0%3A2%3A80b3%3A0%3A3d3&id=o-APCUFwqp9pU4uDV38v71gcyMIIA9sq-Ak9fj-5Z48BUL&itag=18&source=youtube&requiressl=yes&xpc=EgVo2aDSNQ%3D%3D&met=1745846436%2C&mh=tr&mm=31%2C29&mn=sn-nfpuji-1gil%2Csn-1giz7n7l&ms=au%2Crdu&mv=m&mvi=1&pcm2cms=yes&pl=32&rms=au%2Cau&initcwndbps=4433750&bui=AecWEAZs0OQHeW5kiPBvg0AkjuSh2GqGLpensPdvFK-p2KElCiGx7GHfWIjNCUNuYwrQf3D5po71OEUW&vprv=1&svpuc=1&mime=video%2Fmp4&ns=P09nQk3pyhDt3yzwyLO31wcQ&rqh=1&cnr=14&ratebypass=yes&dur=46.068&lmt=1744555934425586&mt=1745845981&fvip=2&lmw=1&c=TVHTML5&sefc=1&txp=8208224&n=YRUKGLJNS9-RcQ&sparams=expire%2Cei%2Cip%2Cid%2Citag%2Csource%2Crequiressl%2Cxpc%2Cbui%2Cvprv%2Csvpuc%2Cmime%2Cns%2Crqh%2Ccnr%2Cratebypass%2Cdur%2Clmt&sig=AJfQdSswRQIhAPSalUoWYRTiKtM9SpYJ1PJtAM83ui2VxL44xHs2f2pkAiB7yfcxBd1ToIIJQxXq8fizk6MbkS7A2zaelC6ad-9y5A%3D%3D&lsparams=met%2Cmh%2Cmm%2Cmn%2Cms%

# THIS PART TRY TO CREATE THE DATASET ON THE FLY WHILE DOWNLOADING THE VIDEO

In [83]:
import os, shutil, subprocess, time, sys
import pandas as pd
import numpy as np
from pydub import AudioSegment
from yt_dlp import YoutubeDL
import cv2

OUTPUT_DIR   = "data_set_try1/processed"       # dossier racine de sortie
CSV_LABELS   = "data_set_try1/dataset/labels_semantic_clusters.csv"
CSV_DATA     = "data_set_try1/dataset/vggsound.csv"
CLUSTER      = '3'
DURATION_SEC = 10

# check ffmpeg
if shutil.which("ffmpeg") is None:
    raise RuntimeError("⚠️ ffmpeg n'est pas installé — indispensable.")
os.makedirs(OUTPUT_DIR, exist_ok=True)


In [84]:
# Cell 2 — Fonctions utilitaires

def sec_to_timestamp(s: int) -> str:
    return time.strftime("%H:%M:%S", time.gmtime(s))

def download_segment(video_id: str, start: int, dur: int, out_dir: str) -> str:
    """Récupère le segment mp4 (audio+vidéo) et renvoie le chemin."""
    url = f"https://www.youtube.com/watch?v={video_id}"
    ydl_opts = {'format': 'best[height<=360][ext=mp4]', 'quiet': True, 'skip_download': True}
    with YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=False)
        stream_url = info['url']
    t0 = sec_to_timestamp(start)
    segment_path = os.path.join(out_dir, "segment.mp4")
    cmd = [
        'ffmpeg', '-ss', t0, '-i', stream_url,
        '-t', str(dur), '-c', 'copy',
        segment_path
    ]
    subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    return segment_path

def download_segment_crop256(video_id: str, start_sec: int, duration: int, dest_folder: str) -> str:
    """
    1) Récupère l’URL du flux MP4 (360p).
    2) Seek + coupe la portion [start_sec, start_sec+duration] via ffmpeg -ss/-t.
    3) Applique un crop 256×256 centré.
    4) Ré-encode la vidéo en H.264 (libx264) et copie l'audio AAC.
    Retourne le chemin du fichier généré.
    """
    os.makedirs(dest_folder, exist_ok=True)
    url = f"https://www.youtube.com/watch?v={video_id}"

    # 1) Récupère l'URL directe du flux MP4
    ydl_opts = {
        'format': 'best[height<=360][ext=mp4]',
        'quiet': True,
        'skip_download': True,
    }
    with YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=False)
        stream_url = info['url']

    # 2) Prépare timestamps et nom de sortie
    t0         = sec_to_timestamp(start_sec)
    out_name   = "segment.mp4"
    out_path   = os.path.join(dest_folder, out_name)

    # 3) ffmpeg : seek rapide (-ss avant -i), durée (-t), crop, encode video, copy audio
    cmd = [
        'ffmpeg',
        '-ss', t0,
        '-i', stream_url,
        '-t', str(duration),
        '-vf', 'crop=256:256:(in_w-256)/2:(in_h-256)/2',
        '-c:v', 'libx264',    # encode video H264
        '-preset', 'veryfast',
        '-crf', '23',
        '-c:a', 'copy',       # copie l'audio tel quel
        out_path
    ]
    subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    return out_path


def analyze_audio_max_timestamp(video_path: str) -> float:
    """Retourne en secondes le point de volume max dans segment.mp4."""
    audio = AudioSegment.from_file(video_path)
    mono  = audio.set_channels(1)
    arr   = np.array(mono.get_array_of_samples())
    idx   = int(np.argmax(np.abs(arr)))
    return idx / mono.frame_rate

def extract_best_thumbnail(video_path: str, ts: float, out_path: str):
    """
    => Prend plusieurs frames autour de ts (±0.5s), calcule la variance
       du Laplacien (proxy pour la netteté), choisit la meilleure,
       fait une égalisation d’histogramme sur la luminance, et sauve.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise RuntimeError(f"Impossible d'ouvrir {video_path}")
    fps      = cap.get(cv2.CAP_PROP_FPS) or 25
    frame_ct = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    duration = frame_ct / fps

    # offsets en secondes autour du pic
    offsets = [-0.5, -0.25, 0, 0.25, 0.5]
    best_score = -1
    best_frame = None

    for off in offsets:
        t = min(max(ts + off, 0), duration)
        cap.set(cv2.CAP_PROP_POS_MSEC, t * 1000)
        ret, frame = cap.read()
        if not ret: 
            continue
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        score = cv2.Laplacian(gray, cv2.CV_64F).var()
        if score > best_score:
            best_score, best_frame = score, frame

    cap.release()

    if best_frame is None:
        raise RuntimeError("Aucune frame extraite pour la thumbnail")

    # égalisation de l'histogramme sur le canal Y de YUV
    yuv = cv2.cvtColor(best_frame, cv2.COLOR_BGR2YUV)
    yuv[:,:,0] = cv2.equalizeHist(yuv[:,:,0])
    enhanced = cv2.cvtColor(yuv, cv2.COLOR_YUV2BGR)

    cv2.imwrite(out_path, enhanced)

def strip_audio(video_path: str, out_path: str):
    """Génère video_no_audio.mp4."""
    cmd = ['ffmpeg', '-i', video_path, '-c', 'copy', '-an', out_path]
    subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

def extract_audio(video_path: str, out_path: str):
    """Génère audio.m4a."""
    cmd = ['ffmpeg', '-i', video_path, '-vn', '-c:a', 'copy', out_path]
    subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

def process_one(vid: str, start: int):
    """Orchestre tout pour un segment."""
    name = f"{vid}_{start}s"
    folder = os.path.join(OUTPUT_DIR, name)
    os.makedirs(folder, exist_ok=True)

    seg = download_segment_crop256(vid, start, DURATION_SEC, folder)
    ts  = analyze_audio_max_timestamp(seg)
    extract_best_thumbnail(seg, ts, os.path.join(folder, "thumbnail.jpg"))
    strip_audio(seg,  os.path.join(folder, "video_no_audio.mp4"))
    extract_audio(seg, os.path.join(folder, "audio.m4a"))


In [85]:
# # Cell 3 — Lecture & filtrage
# labels_df = pd.read_csv(CSV_LABELS, header=0)

# used_labels = labels_df[labels_df['cluster']==CLUSTER]['label'].values

# data_df = pd.read_csv(CSV_DATA, header=None,
#                       names=['video_id','start_sec','label','split'])
# sub_df = data_df[data_df['label'].isin(used_labels)].sample(10, random_state=0)


# Cell 3 — Lecture & filtrage du dataset
labels_df = pd.read_csv(CSV_LABELS, header=None,
                        names=['label','count','cluster'])
used_labels = labels_df[labels_df['cluster']==CLUSTER]['label'].values

data_df = pd.read_csv(CSV_DATA, header=None,
                      names=['video_id','start_sec','label','split'])
sub_df = data_df[data_df['label'].isin(used_labels)].sample(10, random_state=42)


In [86]:
# Cell 4 — Traitement en boucle
for vid, start, *_ in sub_df.itertuples(index=False):
    print(f"🔄 Processing {vid} @ {start}s …")
    try:
        process_one(vid, int(start))
    except Exception as e:
        print(f"⚠️ Erreur pour {vid}@{start}s : {e}")

print("✅ Tout est prêt dans", OUTPUT_DIR)

🔄 Processing 5PVKgUGg0LQ @ 450s …
⚠️ Erreur pour 5PVKgUGg0LQ@450s : Command '['ffmpeg', '-ss', '00:07:30', '-i', 'https://rr1---sn-nfpuji-1gil.googlevideo.com/videoplayback?expire=1745875939&ei=g58PaLHDAZyG6dsPw7jdwQQ&ip=2001%3A620%3A618%3A5c0%3A2%3A80b3%3A0%3A3d3&id=o-AMjaVa6-inKhhYWi_lNVAg6DeOIB60pYvmJrNCH0rkv_&itag=18&source=youtube&requiressl=yes&xpc=EgVo2aDSNQ%3D%3D&met=1745854339%2C&mh=7z&mm=31%2C29&mn=sn-nfpuji-1gil%2Csn-1gi7znek&ms=au%2Crdu&mv=m&mvi=1&pcm2cms=yes&pl=32&rms=au%2Cau&initcwndbps=4430000&bui=AecWEAblI85kL2Z0_jyeYOf2TIdUXhKE_di8pUwTuhgSWEOIgmfMyPmbnfPZjbzv32Zw4dGP8T5yf3D1&vprv=1&svpuc=1&mime=video%2Fmp4&ns=o727fY8-EboKZzse56YIvHkQ&rqh=1&cnr=14&ratebypass=yes&dur=664.926&lmt=1743737791279412&mt=1745853906&fvip=1&lmw=1&c=TVHTML5&sefc=1&txp=5438534&n=SL5K9tWimzub4Q&sparams=expire%2Cei%2Cip%2Cid%2Citag%2Csource%2Crequiressl%2Cxpc%2Cbui%2Cvprv%2Csvpuc%2Cmime%2Cns%2Crqh%2Ccnr%2Cratebypass%2Cdur%2Clmt&sig=AJfQdSswRQIgBwdo-51gClj4RKA3xgSgdJQUYl2O7SjGgAwQFRJM_5YCIQCXtgk0KsgR

ERROR: [youtube] OVBQuUJhtfg: Video unavailable


⚠️ Erreur pour OVBQuUJhtfg@30s : ERROR: [youtube] OVBQuUJhtfg: Video unavailable
🔄 Processing CeTbzVsGfRw @ 30s …
⚠️ Erreur pour CeTbzVsGfRw@30s : Command '['ffmpeg', '-ss', '00:00:30', '-i', 'https://rr1---sn-nfpuji-1gil.googlevideo.com/videoplayback?expire=1745875959&ei=l58PaKy-DubO6dsPirbkkA4&ip=2001%3A620%3A618%3A5c0%3A2%3A80b3%3A0%3A3d3&id=o-AEDAqHzY1eI8IHuwtz07WXJbmYxkouaD9pPBQ9vHVzBU&itag=18&source=youtube&requiressl=yes&xpc=EgVo2aDSNQ%3D%3D&met=1745854359%2C&mh=tr&mm=31%2C29&mn=sn-nfpuji-1gil%2Csn-1gi7znek&ms=au%2Crdu&mv=m&mvi=1&pcm2cms=yes&pl=32&rms=au%2Cau&initcwndbps=4430000&bui=AecWEAbATGoEG9GzMuEx9rAbfYDsnJCjkTabwaNrnsedkpbnsBxWOXbqSyn4-pWtOSlOipvqGUKAcjnB&vprv=1&svpuc=1&mime=video%2Fmp4&ns=7UJT2Inmb4lPg4tOG1GMDQEQ&rqh=1&cnr=14&ratebypass=yes&dur=46.068&lmt=1744555934425586&mt=1745853906&fvip=1&lmw=1&c=TVHTML5&sefc=1&txp=8208224&n=3r5FztHlRrxRKA&sparams=expire%2Cei%2Cip%2Cid%2Citag%2Csource%2Crequiressl%2Cxpc%2Cbui%2Cvprv%2Csvpuc%2Cmime%2Cns%2Crqh%2Ccnr%2Cratebypass%2Cdur%